In [4]:
import netCDF4 as nc
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import xarray as xr
from IO_functions import *
from parameters import *
from statsFunc import crossValidateKfold


Load data

In [5]:
tas1=month_load("C:/Users/predr/Desktop/",mohc_dates,most_dates,years,'tas','ICHEC-EC-EARTH','rcp45','r12i1p1','SMHI-RCA4','v1','2008','12')
tas2=month_load("C:/Users/predr/Desktop/",mohc_dates,most_dates,years,'tas','ICHEC-EC-EARTH','rcp85','r12i1p1','SMHI-RCA4','v1','2008','12')
hurs=month_load("C:/Users/predr/Desktop/",mohc_dates,most_dates,years,'hurs','CNRM-CERFACS-CNRM-CM5','rcp45','r1i1p1','SMHI-RCA4','v1','2008','12')

In [ ]:
x=[]
x=xr.concat([tas1, tas2], dim="time")
#x['tas']

In [ ]:
coef=np.zeros((90,134,2))
adjr2=np.zeros((90,134))
rmse=np.zeros((90,134))

for lat in range(90):
    for lon in range(134):
        tas_cell_month=tas1['tas'][:,lat,lon]
        hurs_cell_month=hurs['hurs'][:,lat,lon]
        coef[lat,lon,:],adjr2[lat,lon],rmse[lat,lon]=crossValidateKfold(tas_cell_month,hurs_cell_month,10)

In [ ]:
rmse

In [22]:
CORDEX_path="C:/Users/predr/Desktop/"
# Parameters
folds= 10
coef=np.zeros((60,90,134,2))
adjr2=np.zeros((60,90,134))
rmse=np.zeros((60,90,134))
month_count=0 #start counting months

for year in range(2006,2010):
    for month in range(1,13):
        tas_all_month=[]
        hurs_all_month=[]
        for gcm in gcms:
            for rcp in rcps:
                for rp in rps:
                    for rcm in rcms:
                        for version in versions:
                            # Load data
                            try:
                                tas=month_load(CORDEX_path,mohc_dates,most_dates,years,'tas',gcm,rcp,rp,rcm,version,str(year),str(month))
                                hurs=month_load(CORDEX_path,mohc_dates,most_dates,years,'hurs',gcm,rcp,rp,rcm,version,str(year),str(month))
                                # Concatenate
                                if any(tas_all_month):
                                    tas_all_month=xr.concat([tas_all_month, tas], dim="time")
                                else:
                                    tas_all_month=tas
                                if any(hurs_all_month):
                                    hurs_all_month=xr.concat([hurs_all_month, hurs], dim="time")
                                else:
                                    hurs_all_month=hurs
                            except Exception as e: 
                                #print(e)
                                x=0
        # Do stuff with monthly data here
        # Regression
        # Might have to do per cell
        #print(tas_all_month.sizes)
        for lat in range(1):
            for lon in range(5):
                tas_cell_month=tas_all_month['tas'][:,lat,lon]
                hurs_cell_month=hurs_all_month['hurs'][:,lat,lon]
                try:
                    coef[month,lat,lon,:],adjr2[month,lat,lon],rmse[month,lat,lon]=crossValidateKfold(tas_cell_month,hurs_cell_month,folds)  
                    print(str(month) + '/' + str(year) + ' finished')     
                except Exception as e: 
                    print(e)
                    pass
        month_count=month_count+1 # increment month

                #print(str(month) + '/' + str(year) + ' finished')


1/2006 finished
1/2006 finished
1/2006 finished
1/2006 finished
1/2006 finished
2/2006 finished
2/2006 finished
2/2006 finished
2/2006 finished
2/2006 finished
3/2006 finished
3/2006 finished
3/2006 finished
3/2006 finished
3/2006 finished
4/2006 finished
4/2006 finished
4/2006 finished
4/2006 finished
4/2006 finished
5/2006 finished
5/2006 finished
5/2006 finished
5/2006 finished
5/2006 finished
6/2006 finished
6/2006 finished
6/2006 finished
6/2006 finished
6/2006 finished
7/2006 finished
7/2006 finished
7/2006 finished
7/2006 finished
7/2006 finished
8/2006 finished
8/2006 finished
8/2006 finished
8/2006 finished
8/2006 finished
9/2006 finished
9/2006 finished
9/2006 finished
9/2006 finished
9/2006 finished
10/2006 finished
10/2006 finished
10/2006 finished
10/2006 finished
10/2006 finished
11/2006 finished
11/2006 finished
11/2006 finished
11/2006 finished
11/2006 finished
12/2006 finished
12/2006 finished
12/2006 finished
12/2006 finished
12/2006 finished
1/2007 finished
1/2007 fi

In [26]:
monthly_mask = xr.open_dataset("C:/Users/predr/Desktop/CNRM-CERFACS-CNRM-CM5/monthly_sample.nc")


<xarray.Dataset>
Dimensions:       (bnds: 2, rlat: 412, rlon: 424, time: 60)
Coordinates:
  * time          (time) datetime64[ns] 2086-01-16T12:00:00 ... 2090-12-16T12...
    lon           (rlat, rlon) float64 ...
    lat           (rlat, rlon) float64 ...
  * rlon          (rlon) float64 -28.38 -28.27 -28.16 ... 17.94 18.05 18.16
  * rlat          (rlat) float64 -23.38 -23.27 -23.16 ... 21.62 21.73 21.84
    height        float64 ...
Dimensions without coordinates: bnds
Data variables:
    time_bnds     (time, bnds) datetime64[ns] ...
    rotated_pole  int32 ...
    tas           (time, rlat, rlon) float32 ...
Attributes:
    CDI:                            Climate Data Interface version 1.9.6 (htt...
    history:                        Mon Dec 21 12:03:26 2020: cdo monmean tas...
    institution:                    Swedish Meteorological and Hydrological I...
    Conventions:                    CF-1.4
    contact:                        rossby.cordex@smhi.se
    creation_date:                  2013-07-19-T11:55:53Z
    experiment:                     RCP4.5
    experiment_id:                  rcp45
    driving_experiment:             CNRM-CERFACS-CNRM-CM5, rcp45, r1i1p1
    driving_model_id:               CNRM-CERFACS-CNRM-CM5
    driving_model_ensemble_member:  r1i1p1
    driving_experiment_name:        rcp45
    frequency:                      mon
    institute_id:                   SMHI
    model_id:                       SMHI-RCA4
    rcm_version_id:                 v1
    project_id:                     CORDEX
    CORDEX_domain:                  EUR-11
    product:                        output
    references:                     http://www.smhi.se/en/Research/Research-d...
    tracking_id:                    c5c1b49b-4022-494f-b6bd-0edaf7f09a68
    rossby_comment:                 201144: CORDEX Europe 0.11 deg | RCA4 v1 ...
    rossby_run_id:                  201144
    rossby_grib_path:               /nobackup/rossby16/rossby/joint_exp/corde...
    CDO:                            Climate Data Operators version 1.9.6 (htt...

In [24]:
foo = xr.DataArray(adjr2, coords=[tas_all_month['time'], tas_all_month['lon'],tas_all_month['lat']], dims=["time", "lon","lat"])

ValueError: conflicting sizes for dimension 'time': length 60 on the data but length 124 on coordinate 'time'